# 6v9z All Trajectory Analysis

## MD Analysis

### Import Modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statistics as s
import MDAnalysis as mda
import random

### Helper Modules

In [ ]:
# Converts 3 letter code to 1 letter code
convert_to_one_letter_dic = {"ALA":"A", "ARG":"R", "ASN":"N", "ASP":"D", "CYS":"C", "GLU":"E", "GLN":"Q", "GLY":"G",\
                            "HSD":"H", "HIS": "H", "HSE":"H", "ILE": "I", "LEU": "L", "LYS":"K", "MET":"M", "PHE":"F", "PRO":"P",\
                            "SER":"S", "THR":"T", "TRP":"W", "TYR":"Y", "VAL":"V"}
def convert_to_one_letter(string):
    amino_acid = str(string[0:3])
    return convert_to_one_letter_dic[amino_acid]

In [ ]:
# Permutation test function
def permutation(pooled,sizeA,sizeB,iterations):
    """
    Function takes pooled samples and randomly shuffles them according to partition size.
    New randommly shuffled partitions are sorted and substracted mean calculated (delta).
    Delta appended to a list. 
    Set x iterations.
    """
    count = 0
    mean_list = []
    while count < iterations:
        np.random.shuffle(pooled)
        new_A = pooled[:sizeA]
        new_B = pooled[-sizeB:]
        delta = s.mean(new_A) - s.mean(new_B)
        mean_list.append(delta)
        count += 1
    return mean_list

In [ ]:
def extract_chain_vals_temp(data):
    """
    Function takes a dataframe and returns substrate or no substrate Lipid_Count values separated by temperature.
    """
    sub_vals_30_deg = []
    sub_vals_37_deg = []
    no_sub_vals_30_deg = []
    no_sub_vals_37_deg = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            if row._3 == 303.15:
                sub_vals_30_deg.append(row.Lipid_Count)
            elif row._3 == 310.15:
                sub_vals_37_deg.append(row.Lipid_Count)
        elif row._2 == 'No':
            if row._3 == 303.15:
                no_sub_vals_30_deg.append(row.Lipid_Count)
            elif row._3 == 310.15:
                no_sub_vals_37_deg.append(row.Lipid_Count)
                
    return sub_vals_30_deg, sub_vals_37_deg, no_sub_vals_30_deg, no_sub_vals_37_deg

In [ ]:
def extract_chain_vals(data):
    """
    Function takes a dataframe and returns substrate or no substrate Lipid_Count values.
    """
    sub_vals = []
    no_sub_vals = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            sub_vals.append(row.Lipid_Count)
        elif row._2 == 'No':
            no_sub_vals.append(row.Lipid_Count)
                
    return sub_vals, no_sub_vals

In [ ]:
def chain_dic_frames(data):
    """
    Function takes a dataframe and returns substrate or no substrate lipid interaction values based on chainID.
    """
    sub_dic = dict()
    nosub_dic = dict()
    for row in data.itertuples():
        if row._1 == "Yes":
            sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_Lipid_Count
        elif row._1 == "No":
            nosub_dic[(row.Chain,row.Joint_Position)] = row.Mean_Lipid_Count
                
    return sub_dic, nosub_dic

In [ ]:
def chain_pair_vals(chain_A,chain_B):
    """
    Function takes dictionary of chain values for chain A and B and returns the difference
    when the amino acids match. Else searches for instances where amino acid is unique to chain A or B.
    """
    A_B_diff = dict()
    unique_A = dict()
    unique_B = dict()
    for k1,v1 in chain_A.items():
        for k2,v2 in chain_B.items():
            if k1 == k2:
                diff = 0
                diff = v1 - v2
                A_B_diff[k1] = diff
            elif k1 not in chain_B.keys():
                unique_A[k1] = v1
    for k3,v3 in chain_B.items():
        if k3 not in chain_A.keys():
            unique_B[k3] = v3
    return A_B_diff, unique_A, unique_B

In [ ]:
def bootstrap_lipid(pooled,n_samples,iterations):
    random.seed(42) # set seed
    count = 0
    mean_diff_list = [] #store median difference values for plotting
    while count < iterations:
        # randomly select samples from the list without replacement
        # calculate mean difference - smoother distn than calc median diff
        np.random.shuffle(pooled) # first shuffle the data
        sample_1 = random.sample(pooled, n_samples)
        sample_2 = random.sample(pooled, n_samples)
        mean_diff = np.mean(sample_1) - np.mean(sample_2)
        mean_diff_list.append(mean_diff)
        count += 1
        
    return mean_diff_list

In [ ]:
def p_val(difference_dic,iterations,permute_results):
    results_array = np.array(permute_results)
    p_val_dict = dict()
    for v1, v2 in difference_dic.items():
        if v2 < 0:
            diffCount = len(np.where(np.sort(results_array) <= v2)[0])
            p_val = float(diffCount)/float(iterations)
            p_val_dict[v1] = p_val
        else:
            diffCount = len(np.where(np.sort(results_array) >= v2)[0])
            p_val = float(diffCount)/float(iterations)
            p_val_dict[v1] = p_val
    return p_val_dict

In [ ]:
def sig_p_val(p_val_dic,sig_level):
    num_tests = int(len(p_val_dic))
    bonferroni = sig_level/num_tests
    sig_vals = dict()
    for key,val in p_val_dic.items():
        if val <= bonferroni:
            sig_vals[key] = val
    return sig_vals

In [ ]:
def extract_lipid_vals_cdl2(data):
    """
    Function takes a dataframe and returns values for CDL2 for either substrate or no substrate.
    """
    cdl2_vals_sub = []
    cdl2_vals_no_sub = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            cdl2_vals_sub.append(row.CDL2)
        elif row._2 == 'No':
            cdl2_vals_no_sub.append(row.CDL2)
    return cdl2_vals_sub, cdl2_vals_no_sub

In [ ]:
def extract_lipid_vals_temp_cdl2(data):
    """
    Function takes a dataframe and returns values for CDL2 for either substrate or no substrate based on temp.
    """
    cdl2_vals_sub_30_deg = []
    cdl2_vals_sub_37_deg = []
    cdl2_vals_no_sub_30_deg = []
    cdl2_vals_no_sub_37_deg = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            if row._3 == 303.15:
                cdl2_vals_sub_30_deg.append(row.CDL2)
            elif row._3 == 310.15:
                cdl2_vals_sub_37_deg.append(row.CDL2)
        elif row._2 == 'No':
            if row._3 == 303.15:
                cdl2_vals_no_sub_30_deg.append(row.CDL2)
            elif row._3 == 310.15:
                cdl2_vals_no_sub_37_deg.append(row.CDL2)
                
    return cdl2_vals_sub_30_deg, cdl2_vals_sub_37_deg, cdl2_vals_no_sub_30_deg, cdl2_vals_no_sub_37_deg

In [ ]:
def lipid_dic_cdl2(data):
    """
    Function takes a dataframe and returns cdl2 interaction values as a dictionary
    by amino acid position and chain ID
    """
    cdl2_sub_dic = dict()
    cdl2_no_sub_dic = dict()
    for row in data.itertuples():
        if row._1 == 'Yes':
            cdl2_sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_CDL2_Count
        elif row._1 == 'No':
            cdl2_no_sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_CDL2_Count
            
    return cdl2_sub_dic, cdl2_no_sub_dic

In [ ]:
def extract_lipid_vals_pope_popg(data):
    """
    Function takes a dataframe and returns values for POPE/POPG for either substrate or no substrate.
    """
    pope_popg_vals_sub = []
    pope_popg_vals_no_sub = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            pope_popg_vals_sub.append(row.POPE + row.POPG)
        elif row._2 == 'No':
            pope_popg_vals_no_sub.append(row.POPE + row.POPG)
    return pope_popg_vals_sub, pope_popg_vals_no_sub

In [ ]:
def extract_lipid_vals_temp_pope_popg(data):
    """
    Function takes a dataframe and returns values for POPE/POPG for either substrate or no substrate based on temp.
    """
    pope_popg_vals_sub_30_deg = []
    pope_popg_vals_sub_37_deg = []
    pope_popg_vals_no_sub_30_deg = []
    pope_popg_vals_no_sub_37_deg = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            if row._3 == 303.15:
                pope_popg_vals_sub_30_deg.append(row.POPE + row.POPG)
            elif row._3 == 310.15:
                pope_popg_vals_sub_37_deg.append(row.POPE + row.POPG)
        elif row._2 == 'No':
            if row._3 == 303.15:
                pope_popg_vals_no_sub_30_deg.append(row.POPE + row.POPG)
            elif row._3 == 310.15:
                pope_popg_vals_no_sub_37_deg.append(row.POPE + row.POPG)
                
    return pope_popg_vals_sub_30_deg, pope_popg_vals_sub_37_deg, pope_popg_vals_no_sub_30_deg, pope_popg_vals_no_sub_37_deg

In [ ]:
def lipid_dic_pope_popg(data):
    """
    Function takes a dataframe and returns cdl2 interaction values as a dictionary
    by amino acid position and chain ID
    """
    pope_popg_sub_dic = dict()
    pope_popg_no_sub_dic = dict()
    for row in data.itertuples():
        if row._1 == 'Yes':
            pope_popg_sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_POPE_POPG_Count
        elif row._1 == 'No':
            pope_popg_no_sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_POPE_POPG_Count
            
    return pope_popg_sub_dic, pope_popg_no_sub_dic

In [ ]:
def extract_sub_vals_clust(data):
    """
    Function takes a dataframe and returns substrate or no substrate average median count values.
    """
    sub_vals = []
    no_sub_vals = []
    for row in data.itertuples():
        if row.Substrate == 'yes':
            sub_vals.append(row.Scaled_Median_Count)
        elif row.Substrate  == 'no':
            no_sub_vals.append(row.Scaled_Median_Count)
                
    return sub_vals, no_sub_vals

### Plots for thesis - set colours

In [ ]:
# Select palette
palette = sns.color_palette("deep")

In [ ]:
blue = sns.color_palette("deep")[0]
orange = sns.color_palette("deep")[1]
green = sns.color_palette("deep")[2]
red = sns.color_palette("deep")[3]
purple = sns.color_palette("deep")[4]
brown = sns.color_palette("deep")[5]
pink = sns.color_palette("deep")[6]
grey = sns.color_palette("deep")[7]
gold = sns.color_palette("deep")[8]
turqoise = sns.color_palette("deep")[9]

### Define Universes

In [ ]:
# Read trajectory - ignore warning as this is coarse grain
u1 = mda.Universe("6v9z_sub_30_deg_traj_1_chainAB.pdb","6v9z_sub_30_deg_skip1000_chainAB_traj_1.xtc")
u2 = mda.Universe("6v9z_sub_30_deg_traj_2_chainAB.pdb","6v9z_sub_30_deg_skip1000_chainAB_traj_2.xtc")
u3 = mda.Universe("6v9z_sub_37_deg_traj_1_chainAB.pdb","6v9z_sub_37_deg_skip1000_chainAB_traj_1.xtc")
u4 = mda.Universe("6v9z_sub_37_deg_traj_2_chainAB.pdb","6v9z_sub_37_deg_skip1000_chainAB_traj_2.xtc")
u5 = mda.Universe("6v9z_nosub_30_deg_traj_1_chainAB.pdb","6v9z_nosub_30_deg_skip1000_chainAB_traj_1.xtc")
u6 = mda.Universe("6v9z_nosub_30_deg_traj_2_chainAB.pdb","6v9z_nosub_30_deg_skip1000_chainAB_traj_2.xtc")
u7 = mda.Universe("6v9z_nosub_37_deg_traj_1_chainAB.pdb","6v9z_nosub_37_deg_skip1000_chainAB_traj_1.xtc")
u8 = mda.Universe("6v9z_nosub_37_deg_traj_2_chainAB.pdb","6v9z_nosub_37_deg_skip1000_chainAB_traj_2.xtc")

In [ ]:
labels = ["u1","u2","u3","u4","u5","u6","u7","u8"]

In [ ]:
# Check length of the trajectories
print(len(u1.trajectory), len(u2.trajectory), len(u3.trajectory), len(u4.trajectory), len(u5.trajectory), len(u6.trajectory), len(u7.trajectory), len(u8.trajectory))

### Ensemble Cluster Analysis

This analysis only works with preselected atoms in trajectories. Only reduce number of frames to speed up calculation.

In [ ]:
# import modules
from MDAnalysis.analysis import encore
from MDAnalysis.analysis.encore.clustering import ClusteringMethod as clm

In [ ]:
# Cluster
ces0, details0 = encore.ces(ensembles=[u1,u2,u3,u4,u5,u6,u7,u8], select="type B")

In [ ]:
# Print cluster info
cluster_collection = details0['clustering'][0]
print(type(cluster_collection))
print('We have found {} clusters'.format(len(cluster_collection)))

In [ ]:
# k-means clustering
km1 = clm.KMeans(16,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default

km2 = clm.KMeans(8,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default

km3 = clm.KMeans(4,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default

km4 = clm.KMeans(2,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default



In [ ]:
ces2, details2 = encore.ces([u1,u2,u3,u4,u5, u6, u7, u8],
                         select='type B',
                         clustering_method=[km1, km2, km3, km4],
                         ncores=4)
print(len(ces2), len(details2['clustering']))

In [ ]:
# Plot clustering
titles = ['Kmeans 16 clusters', 'Kmeans 8 clusters', 'Kmeans 4 clusters', 'Kmeans 2 clusters']
fig2, axes = plt.subplots(1, 4, sharey=True, figsize=(15, 4))
for i, (data, title) in enumerate(zip(ces2, titles)):
    imi = axes[i].imshow(data, vmax=np.log(2), vmin=0)
    axes[i].set_xticks(np.arange(8))
    axes[i].set_xticklabels(labels)
    axes[i].set_title(title)
plt.yticks(np.arange(8), labels)
cbar2 = fig2.colorbar(imi, ax=axes.ravel().tolist())
cbar2.set_label('Jensen-Shannon divergence')
plt.savefig("Ensemble Cluster K means All Trajectories 4.png", dpi = 300)

In [ ]:
#estimate errors
avgs, stds = encore.ces([u1, u2, u3, u4, u5, u6, u7, u8],
                         select='type B',
                         clustering_method=km3,
                         estimate_error=True,
                         ncores=4)

In [ ]:
avgs

In [ ]:
stds

## Protein Lipid Contacts

##### Raw Data 

In [ ]:
# Read in csv's
df_1 = pd.read_csv("data/6v9z_sub_30_deg_traj_1_protein_lipid_contact.csv")
df_2 = pd.read_csv("data/6v9z_sub_30_deg_traj_2_protein_lipid_contact.csv")
df_3 = pd.read_csv("data/6v9z_sub_37_deg_traj_1_protein_lipid_contact.csv")
df_4 = pd.read_csv("data/6v9z_sub_37_deg_traj_2_protein_lipid_contact.csv")
df_5 = pd.read_csv("data/6v9z_nosub_30_deg_traj_1_protein_lipid_contact.csv")
df_6 = pd.read_csv("data/6v9z_nosub_30_deg_traj_2_protein_lipid_contact.csv")
df_7 = pd.read_csv("data/6v9z_nosub_37_deg_traj_1_protein_lipid_contact.csv")
df_8 = pd.read_csv("data/6v9z_nosub_37_deg_traj_2_protein_lipid_contact.csv")

#### Combine Data

In [ ]:
frames = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8]
combined_df = pd.concat(frames, ignore_index=True)

In [ ]:
#Add one letter code back in
combined_df["Protein_Code"] = combined_df["Protein_Type"].apply(convert_to_one_letter)

In [ ]:
#Get actual resids to match structure
combined_df["Actual_Protein_ResID"] = combined_df["Protein_ResID"] + 7

In [ ]:
combined_df["Joint_Position"] = combined_df["Protein_Code"].astype(str) + combined_df["Actual_Protein_ResID"].astype(str)

In [ ]:
# First group by ID to sum appropriate lipid values
group_contact = combined_df.groupby(["ID","Substrate?","Temperature(K)","Chain","Actual_Protein_ResID","Joint_Position", "Lipid_Type"])["Fraction_Frames"].sum().unstack()

In [ ]:
#Reset index to remove multi-indexing
group_contact.reset_index(inplace = True)

In [ ]:
#Fill in NaNs with 0s
group_contact["CDL2"].fillna(value = 0, inplace = True)
group_contact["POPE"].fillna(value = 0, inplace = True)
group_contact["POPG"].fillna(value = 0, inplace = True)

In [ ]:
#sort values by protein position
group_contact.sort_values("Actual_Protein_ResID", axis = 0, ascending = True, inplace = True) #sort values by protein position

In [ ]:
# Add in sub columns for plotting
group_contact["Lipid_Count"] = group_contact["CDL2"] + group_contact["POPE"] + group_contact["POPG"]
group_contact["POPE_POPG"] = group_contact["POPE"] + group_contact["POPG"]
group_contact["Substrate?_Temperature"] = group_contact["Substrate?"].astype(str) + "_" + group_contact["Temperature(K)"].astype(str)
group_contact["Substrate?_Temperature_Chain"] = group_contact["Substrate?"].astype(str) + "_" + group_contact["Temperature(K)"].astype(str) + "_" + group_contact["Chain"].astype(str)
group_contact["Substrate?_Chain"] = group_contact["Substrate?"].astype(str) + "_" + group_contact["Chain"].astype(str)

In [ ]:
# Aggregate data - only separate by presence of substrate
group_contact_cdl2_lipid = group_contact.groupby(["ID","Temperature(K)","Substrate?","Chain","Actual_Protein_ResID","Joint_Position"])["CDL2"].mean().unstack()
group_contact_pope_popg_lipid = group_contact.groupby(["ID","Temperature(K)","Substrate?","Chain","Actual_Protein_ResID","Joint_Position"])["POPE_POPG"].mean().unstack()

In [ ]:
 #Reset index to remove multi-indexing
group_contact_cdl2_lipid.reset_index(inplace = True)
group_contact_pope_popg_lipid.reset_index(inplace = True)

In [ ]:
# remove extra headers - cardiolipin
head_list_cdl2_lipid = group_contact_cdl2_lipid.columns.tolist()
head_list_cdl2_lipid.remove('ID')
head_list_cdl2_lipid.remove('Temperature(K)')
head_list_cdl2_lipid.remove('Substrate?')
head_list_cdl2_lipid.remove('Chain')
head_list_cdl2_lipid.remove('Actual_Protein_ResID')

In [ ]:
# remove extra headers - pope/popg
head_list_pope_popg_lipid = group_contact_pope_popg_lipid.columns.tolist()
head_list_pope_popg_lipid.remove('ID')
head_list_pope_popg_lipid.remove('Temperature(K)')
head_list_pope_popg_lipid.remove('Substrate?')
head_list_pope_popg_lipid.remove('Chain')
head_list_pope_popg_lipid.remove('Actual_Protein_ResID')

In [ ]:
# Apply melt
melt_group_contact_cdl2 = group_contact_cdl2_lipid.melt(id_vars=['ID','Temperature(K)','Substrate?','Chain','Actual_Protein_ResID'],value_vars=head_list_cdl2_lipid,value_name="Mean_CDL2_Count")
melt_group_contact_pope_popg = group_contact_pope_popg_lipid.melt(id_vars=['ID','Temperature(K)','Substrate?','Chain','Actual_Protein_ResID'],value_vars=head_list_pope_popg_lipid,value_name="Mean_POPE_POPG_Count")

In [ ]:
# Remove nan values
melt_group_contact_cdl2.dropna(inplace = True)
melt_group_contact_pope_popg.dropna(inplace = True)

In [ ]:
# Sort for plotting - by protein residue number
melt_group_contact_cdl2.sort_values(["Actual_Protein_ResID","Chain"], axis = 0, ascending = True, inplace = True) #sort values by protein position
melt_group_contact_pope_popg.sort_values(["Actual_Protein_ResID","Chain"], axis = 0, ascending = True, inplace = True) #sort values by protein position

In [ ]:
# Separate by substrate and chain ID
melt_group_contact_cdl2["Substrate?_Chain"] = melt_group_contact_cdl2["Substrate?"].astype(str) + "_" +  melt_group_contact_cdl2["Chain"].astype(str)
melt_group_contact_pope_popg["Substrate?_Chain"] = melt_group_contact_pope_popg["Substrate?"].astype(str) + "_" +  melt_group_contact_pope_popg["Chain"].astype(str)

In [ ]:
# Remove small values
melt_group_contact_cdl2_remove = melt_group_contact_cdl2[melt_group_contact_cdl2["Mean_CDL2_Count"]>0.01]
melt_group_contact_pope_popg_remove = melt_group_contact_pope_popg[melt_group_contact_pope_popg["Mean_POPE_POPG_Count"]>0.01]

Plotting when small values are removed will result in an incorrect plot. We will first define the positions which contain significantly large values, and remove others from the original "melt_group_contact_{lipid}" dataframe.

In [ ]:
# Define dictionary to store positions with significant cdl2 contacts
x_axis_headers_cdl2 = []
for row in melt_group_contact_cdl2_remove.itertuples():
    if row.Joint_Position not in x_axis_headers_cdl2:
        x_axis_headers_cdl2.append(row.Joint_Position)

In [ ]:
# Define dictionary to store positions with significant pope/popg contacts
x_axis_headers_popeg = []
for row in melt_group_contact_pope_popg_remove.itertuples():
    if row.Joint_Position not in x_axis_headers_popeg:
        x_axis_headers_popeg.append(row.Joint_Position)

In [ ]:
# Use these to make a new dataframe - cdl2
cdl2_ID_list = []
cdl2_temp_list = []
cdl2_sub_list = []
cdl2_chain_list = []
cdl2_resid_list = []
cdl2_joint_position_list = []
cdl2_list = []
cdl2_label_list = []

In [ ]:
# Use these to make a new dataframe - pope/popg
pope_popg_ID_list = []
pope_popg_temp_list = []
pope_popg_sub_list = []
pope_popg_chain_list = []
pope_popg_resid_list = []
pope_popg_joint_position_list = []
popeg_list = []
pope_popg_label_list = []

In [ ]:
# Loop over original dataframe and extract data from positions with significant cdl2 contacts
for row in melt_group_contact_cdl2.itertuples():
    if row.Joint_Position in x_axis_headers_cdl2:
        cdl2_ID_list.append(row.ID)
        cdl2_temp_list.append(row._2)
        cdl2_sub_list.append(row._3)
        cdl2_chain_list.append(row.Chain)
        cdl2_resid_list.append(row.Actual_Protein_ResID)
        cdl2_joint_position_list.append(row.Joint_Position)
        cdl2_list.append(row.Mean_CDL2_Count)
        cdl2_label_list.append(row._8)

In [ ]:
# Loop over original dataframe and extract data from positions with significant pope/popg contacts
for row in melt_group_contact_pope_popg.itertuples():
    if row.Joint_Position in x_axis_headers_popeg:
        pope_popg_ID_list.append(row.ID)
        pope_popg_temp_list.append(row._2)
        pope_popg_sub_list.append(row._3)
        pope_popg_chain_list.append(row.Chain)
        pope_popg_resid_list.append(row.Actual_Protein_ResID)
        pope_popg_joint_position_list.append(row.Joint_Position)
        popeg_list.append(row.Mean_POPE_POPG_Count)
        pope_popg_label_list.append(row._8)

In [ ]:
# Make a dictionary of the lists with column headings - cdl2
data_cdl2 = {'ID':cdl2_ID_list, 'Temperature(K)':cdl2_temp_list, 'Substrate?':cdl2_sub_list,'Chain':cdl2_chain_list,
        'Actual_Protein_ResID':cdl2_resid_list,'Joint_Position':cdl2_joint_position_list,
        'Mean_CDL2_Count':cdl2_list, 'Substrate?_Chain':cdl2_label_list}

In [ ]:
# Make a dictionary of the lists with column headings - pope/popg
data_pope_popg = {'ID':pope_popg_ID_list, 'Temperature(K)':pope_popg_temp_list, 'Substrate?':pope_popg_sub_list,
         'Chain':pope_popg_chain_list, 'Actual_Protein_ResID':pope_popg_resid_list,
         'Joint_Position':pope_popg_joint_position_list, 'Mean_POPE_POPG_Count':popeg_list, 
         'Substrate?_Chain':pope_popg_label_list}

In [ ]:
# Use dictionaries to make new dataframes
cdl2_new = pd.DataFrame.from_dict(data_cdl2)
popeg_new = pd.DataFrame.from_dict(data_pope_popg)

##### Plots for thesis

In [ ]:
# Palette dictionary
palette_dic = {"Yes_A":blue, "No_A":orange,"Yes_B":green,"No_B":red}

In [ ]:
# Plot using seaborn - cdl2
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
melt_group_contact_cdl2_fig = sns.barplot(x = "Joint_Position", y = "Mean_CDL2_Count", 
                                          hue = "Substrate?_Chain", 
                                          hue_order=["Yes_A","No_A","Yes_B","No_B"], 
                                          data = cdl2_new, palette=palette_dic, 
                                          estimator=np.mean,ci=90, ax=ax).get_figure()
plt.xlabel("ResID")
plt.ylabel("CDL2 Fraction Frames")
plt.legend(loc='upper left');
melt_group_contact_cdl2_fig.savefig("Mean CDL2 Count by Substrate and Chain Relabel Remove correct.png", dpi = 300)

In [ ]:
# Plot using seaborn -pope/popg
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
melt_group_contact_popeg_fig = sns.barplot(x = "Joint_Position", y = "Mean_POPE_POPG_Count",
                                           hue = "Substrate?_Chain", 
                                           hue_order=["Yes_A","No_A","Yes_B","No_B"], 
                                           data = popeg_new, palette=palette_dic, 
                                           estimator=np.mean,ci=90, ax=ax).get_figure()
plt.xlabel("ResID")
plt.ylabel("POPE+POPG Fraction Frames")
plt.legend(loc='upper left');
melt_group_contact_popeg_fig.savefig("Mean POPE POPG Count by Substrate and Chain Relabel Remove correct.png", dpi = 300)

In [ ]:
#remove small values for plotting - exploratory
group_contact_lipid_count = group_contact[group_contact["Lipid_Count"] > 0.01]
group_contact_cdl2 = group_contact[group_contact["CDL2"] > 0.01]
group_contact_pope = group_contact[group_contact["POPE"] > 0.01]
group_contact_popg = group_contact[group_contact["POPG"] > 0.01]

##### Exploratory Plots

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_lip_fig = sns.barplot(x = "Joint_Position", y = "Lipid_Count", hue = "Substrate?_Chain", data = group_contact, ax=ax).get_figure();
group_contact_lip_fig.savefig("Lipid Count Substrate and Chain All Data.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_lip_fig = sns.barplot(x = "Joint_Position", y = "Lipid_Count", hue = "Substrate?_Temperature", data = group_contact_lipid_count, ax=ax).get_figure();
group_contact_lip_fig.savefig("Lipid Count Substrate and Temp.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_sub_fig = sns.barplot(x = "Joint_Position", y = "Lipid_Count", hue = "Substrate?", data = group_contact_lipid_count, ax=ax).get_figure();
group_contact_sub_fig.savefig("Lipid Count Substrate.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_temp_fig = sns.barplot(x = "Joint_Position", y = "Lipid_Count", hue = "Temperature(K)", data = group_contact_lipid_count, ax=ax).get_figure();
group_contact_temp_fig.savefig("Lipid Count Temperature.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_cdl2_fig = sns.barplot(x = "Joint_Position", y = "CDL2", hue = "Substrate?_Chain", data = group_contact_cdl2, ax=ax).get_figure();
group_contact_cdl2_fig.savefig("CDL2 Count Substrate and Chain Remove Small Values.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_pope_fig = sns.barplot(x = "Joint_Position", y = "POPE", hue = "Substrate?_Chain", data = group_contact_pope, ax=ax).get_figure();
group_contact_pope_fig.savefig("POPE Count Substrate and Chain.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_popg_fig = sns.barplot(x = "Joint_Position", y = "POPG", hue = "Temperature(K)", data = group_contact_popg, ax=ax).get_figure();
group_contact_popg_fig.savefig("POPG Count Temperature.png", dpi = 300)

# Significance Tests for Protein-Lipid Interactions

#### Check if CDL2 Preference Between Substrate vs No Substrate

Keep in mind this result is using all the data! Don't want to do test on data with small points removed - give biased results & skews the test. I made a plot with all the data for substrate & chain to confirm mean plot is correct. Also keep in mind that with the combined temps a bit of power is removed so we're only really detecting the very large differences in sub vs no sub split by chain - this is helpful to only focus on most promising leads. I checked the plots & confirmed they looked reasonable - only very large differences picked up as significant.

In [ ]:
# Extract lipid values based on substrate presence + temp - full data
cdl2_sub_30, cdl2_sub_37, cdl2_nosub_30, cdl2_nosub_37 = extract_lipid_vals_temp_cdl2(group_contact)

In [ ]:
#Extract values from original dataframe - full data
cdl2_sub, cdl2_no_sub = extract_lipid_vals_cdl2(group_contact)

In [ ]:
# Check paritions for sub + temp
len(cdl2_sub_30)

length cdl2_sub_30 - 243\
length cdl2_sub_37 - 233\
length cdl2_nosub_30 - 241\
length cdl2_nosub_37 - 241


Set these partitions

In [ ]:
# Try bootstrap test for substrate vs no substrate CDL2 at 303.15 K
pooled_cdl2_30_boot = list(np.hstack([cdl2_sub_30,cdl2_nosub_30]))
n_samples = 48
iterations = 1000000
boot_test_cdl2_30 = bootstrap_lipid(pooled_cdl2_30_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram sub vs no sub CDL2 preference at 303.15 K
plt.hist(boot_test_cdl2_30,bins=100);
plt.title("Bootstrap Test CDL2 Preference Substrate vs No Substrate at 303.15 K")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram CDL2 Preference Substrate 30 Deg", dpi=150)

In [ ]:
# Try bootstrap test for substrate vs no substrate CDL2 at 310.15 K
pooled_cdl2_37_boot = list(np.hstack([cdl2_sub_37,cdl2_nosub_37]))
n_samples = 48
iterations = 1000000
boot_test_cdl2_37 = bootstrap_lipid(pooled_cdl2_37_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram sub vs no sub CDL2 preference at 303.15 K
plt.hist(boot_test_cdl2_37,bins=100);
plt.title("Bootstrap Test CDL2 Preference Substrate vs No Substrate at 310.15 K")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram CDL2 Preference Substrate 37 Deg", dpi=150)

Identical distributions for CDl2 at 303.15 and 310.15 K - can use same distribution and combine temps!

In [ ]:
# Check paritions for sub
len(cdl2_no_sub)

472 values for CDL2 Substrate\
482 values for CDL2 No Substrate

Set these partitions

In [ ]:
# Try bootstrap test for substrate vs no substrate CDL2
pooled_cdl2_boot = list(np.hstack([cdl2_sub,cdl2_no_sub]))
n_samples = 96
iterations = 1000000
boot_test_cdl2 = bootstrap_lipid(pooled_cdl2_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram
plt.hist(boot_test_cdl2,bins=100);
plt.title("Bootstrap Test CDL2 Preference Substrate vs No Substrate")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Boostrap Test Histogram CDL2 Preference Substrate", dpi=150)

In [ ]:
# Separate out substrate and no substrate for cdl2
dic_cdl2_sub, dic_cdl2_no_sub = lipid_dic_cdl2(melt_group_contact_cdl2)

In [ ]:
# Calculate differences in cdl2 values based on protein residue
cdl2_sub_diff, cdl2_sub_unique, cdl2_no_sub_unique = chain_pair_vals(dic_cdl2_sub,dic_cdl2_no_sub)

In [ ]:
#Combine all three dicts to one dictionary
total_cdl2_diff = {**cdl2_sub_diff,**cdl2_sub_unique,**cdl2_no_sub_unique}

In [ ]:
#Actual difference between substrate vs no substrate CDL2 preference
plt.hist(total_cdl2_diff.values(),bins=100);
plt.title("Actual Substrate vs No Substrate CDL2 Preference")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
#plt.savefig("Histogram Substrate vs No Substrate CDL2 Preference", dpi=150)

In [ ]:
#Calculate p-values using bootstrap test distribution for CDL2 preference
cdl2_diff_p_val_boot = p_val(total_cdl2_diff,iterations,boot_test_cdl2)

In [ ]:
# Convert p-values dictionary to dataframe for saving
cdl2_diff_p_val_bootseries = pd.Series(cdl2_diff_p_val_boot, name='p-value')
cdl2_diff_p_val_bootseries.index.name = "Amino Acid"
cdl2_diff_p_val_bootseries.reset_index()

In [ ]:
#save p-values!
cdl2_diff_p_val_bootseries.to_csv('CDL2 Substrate vs No Substrate p-values bootstrap.csv')

In [ ]:
#Filter for significant p-values for CDL2 preference
cdl2_diff_p_val_sig_boot = sig_p_val(cdl2_diff_p_val_boot,0.05)

17 amino acids with a statistically significant (1%) difference in CDL2 preference between substrate and no substrate based on the bootstrap test:

Chain A:\
Q24, T144 (5% sig level), N146, T161, S299, N320, N325, R328 (5% sig level), N412 (5% sig level), Q443, T444, N450

Chain B:\
K1, K2, Q24, S98, K100, T105, S300 (5% sig level), K321, Q443

CDL2 favoured when substrate present (Chain A):\
Q24, T144, N146, T161, S299, N325, R328, Q443, T444, N450

CDL2 favoured when substrate not present (Chain A):\
N320, N412

CDL2 favoured when substrate present (Chain B):\
S300, K321, Q443

CDL2 favoured when substrate not present (Chain B):\
K1, K2, Q24, S98, K100, T105


#### POPE/POPG Preference Substrate vs No Substrate

In [ ]:
# Extract lipid values based on substrate presence + temp - full data
pope_popg_sub_30, pope_popg_sub_37, pope_popg_nosub_30, pope_popg_nosub_37 = extract_lipid_vals_temp_pope_popg(group_contact)

In [ ]:
#Extract values from original dataframe - full data
pope_popg_sub, pope_popg_no_sub = extract_lipid_vals_pope_popg(group_contact)

In [ ]:
len(pope_popg_nosub_30)

length pope_popg_sub_30 - 242\
length pope_popg_sub_37 - 230\
length pope_popg_nosub_30 - 241\
length pope_popg_nosub_37 - 241

Set these partitions


In [ ]:
# Try bootstrap test for POPE/POPG preference substrate vs no substrate at 303.15 K
pooled_pope_popg_boot = list(np.hstack([pope_popg_sub_30,pope_popg_nosub_30]))
n_samples = 48
iterations = 1000000
boot_test_pope_popg_30 = bootstrap_lipid(pooled_pope_popg_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram at 303.15 K
plt.hist(boot_test_pope_popg_30,bins=100);
plt.title("Bootstrap Test POPE/POPG Preference Substrate vs No Substrate at 303.15 K")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram Substrate vs No Substrate POPE POPG Preference 30 Deg", dpi=150)

In [ ]:
# Try bootstrap test for POPE/POPG preference substrate vs no substrate at 310.15 K
pooled_pope_popg_boot_37 = list(np.hstack([pope_popg_sub_37,pope_popg_nosub_37]))
n_samples = 48
iterations = 1000000
boot_test_pope_popg_37 = bootstrap_lipid(pooled_pope_popg_boot_37,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram at 310.15 K
plt.hist(boot_test_pope_popg_37,bins=100);
plt.title("Bootstrap Test POPE/POPG Preference Substrate vs No Substrate at 310.15 K")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram Substrate vs No Substrate POPE POPG Preference 37 Deg", dpi=150)

Identical distributions for CDl2 at 303.15 and 310.15 K - can use same distribution and combine temps!

In [ ]:
len(pope_popg_sub)

pope_popg_sub - 472\
pope_popg_no_sub - 482

Set these partitions

In [ ]:
# Get bootstrap test for substrate vs no substrate POPE/POPG preference
pooled_pope_popg_boot = list(np.hstack([pope_popg_sub,pope_popg_no_sub]))
n_samples = 96
iterations = 1000000
boot_test_pope_popg = bootstrap_lipid(pooled_pope_popg_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram
plt.hist(boot_test_pope_popg,bins=100);
plt.title("Boostrap Test POPE/POPG Preference Substrate vs No Substrate")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram POPE and POPG Preference Substrate", dpi=150)

In [ ]:
# Separate out substrate and no substrate for pope/popg
dic_pope_popg_sub, dic_pope_popg_no_sub = lipid_dic_pope_popg(melt_group_contact_pope_popg)

In [ ]:
# Calculate differences in pope/popg values based on protein residue
pope_popg_sub_diff, pope_popg_sub_unique, pope_popg_no_sub_unique = chain_pair_vals(dic_pope_popg_sub,dic_pope_popg_no_sub)

In [ ]:
#Combine all three dicts to one dictionary
total_pope_popg_diff = {**pope_popg_sub_diff,**pope_popg_sub_unique,**pope_popg_no_sub_unique}

In [ ]:
#Actual difference between POPE/POPG preference substrate vs no substrate
plt.hist(total_pope_popg_diff.values(),bins=100);
plt.title("Actual Substrate vs No Substrate POPE/POPG Preference")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
#plt.savefig("Histogram Substrate vs No Substrate POPE POPG Preference", dpi=150)

In [ ]:
#Calculate p-values using bootstrap test distribution for POPE/POPG
pope_popg_diff_p_val_boot = p_val(total_pope_popg_diff,iterations,boot_test_pope_popg)

In [ ]:
# Convert p-values dictionary to dataframe for saving
pope_popg_diff_p_val_series_boot = pd.Series(pope_popg_diff_p_val_boot, name='p-value')
pope_popg_diff_p_val_series_boot.index.name = "Amino Acid"
pope_popg_diff_p_val_series_boot.reset_index()

In [ ]:
#save p-values!
pope_popg_diff_p_val_series_boot.to_csv('Substrate vs No Substrate POPE POPG Preference p-values bootstrap.csv')

In [ ]:
#Filter for significant p-values for bootstrap test
pope_popg_diff_p_val_sig_boot = sig_p_val(pope_popg_diff_p_val_boot,0.05)

Set of 15 amino acids with a significant difference (1%) in POPE/POPG preference between substrate vs no substrate for boostrap test:

Chain A:\
T144, N146, T161, S300, N325, N407, Q443, N450

Chain B:\
T105, T144 (5% sig level), Q145 (5% sig level), T161, N197, T317, N320, N325 (5% sig level), Q439, Q443

In favour of presence of substrate (Chain A):\
S300, N325, N407

In favour of presence of substrate (Chain B):\
T105, T144, T161

In favour of no substrate presence (Chain A):\
T144, N146, T161, Q443, N450

In favour of no substrate presence (Chain B):\
Q145, N197, T317, N320, N325, Q439, Q443


## Protein-Substrate Contacts

### Import Data

In [ ]:
sub_df_1 = pd.read_csv("data/protein_substrate_contacts_30_deg_traj_1.csv")
sub_df_2 = pd.read_csv("data/protein_substrate_contacts_30_deg_traj_2.csv")
sub_df_3 = pd.read_csv("data/protein_substrate_contacts_37_deg_traj_1.csv")
sub_df_4 = pd.read_csv("data/protein_substrate_contacts_37_deg_traj_2.csv")

In [ ]:
sub_frames = [sub_df_1,sub_df_2,sub_df_3,sub_df_4]
df_substrate = pd.concat(sub_frames, ignore_index=True)

In [ ]:
# Set protein residue number so it matches the structure!
df_substrate["Actual Acceptor Protein ResID"] = df_substrate["Acceptor Protein ResID"] + 7

### Extract Amino Acid Type

In [ ]:
df_substrate["Substrate Joint Position"] = df_substrate["Substrate Protein Type"].astype(str) + df_substrate["Substrate Protein ResID"].astype(str)
df_substrate["Acceptor Joint Position"] = df_substrate["Acceptor Protein Type"].astype(str) + df_substrate["Actual Acceptor Protein ResID"].astype(str)

In [ ]:
#sort values by protein position
df_substrate.sort_values("Actual Acceptor Protein ResID", axis = 0, ascending = True, inplace = True)

In [ ]:
# Calculate mean fraction frames
group_sub_contact = df_substrate.groupby(["ID","Temperature (K)","Acceptor ChainID", "Actual Acceptor Protein ResID","Acceptor Joint Position","Substrate ChainID","Substrate Joint Position"])["Fraction_Frames"].mean().unstack()

In [ ]:
 #Reset index to remove multi-indexing
group_sub_contact.reset_index(inplace = True)

In [ ]:
#sort values by protein position
group_sub_contact.sort_values("Actual Acceptor Protein ResID", axis = 0, ascending = True, inplace = True) 

In [ ]:
# Filter Interactions by chain (C or D substrate)
chain_C_filter = group_sub_contact[group_sub_contact["Substrate ChainID"] == "C"].copy()
chain_D_filter = group_sub_contact[group_sub_contact["Substrate ChainID"] == "D"].copy()

In [ ]:
# Apply melt for later plotting - chain C
c_melt = chain_C_filter.melt(id_vars = ["ID","Temperature (K)","Actual Acceptor Protein ResID", "Acceptor Joint Position", "Acceptor ChainID"], value_vars = ["N1","I2","G3","R4","E5","T7","D8",
                                                                                                   "E9","E10","M12","E13","M14","T15",
                                                                                                   "G16","G17","S18","T19","F20","S21",
                                                                                                   "I22","Q23"],
                                                                                                   value_name="Fraction_Frames")

In [ ]:
# Apply melt for later plotting - chain D
d_melt = chain_D_filter.melt(id_vars = ["ID","Temperature (K)","Actual Acceptor Protein ResID", "Acceptor Joint Position", "Acceptor ChainID"], value_vars = ["N1","I2","G3","R4","E5","T7","D8",
                                                                                                   "E9","E10","M12","E13","M14","T15",
                                                                                                   "G16","G17","S18","T19"],
                                                                                                   value_name="Fraction_Frames")

In [ ]:
#add temp + chainID column
c_melt["Temperature_AcceptorChainID"] = c_melt["Temperature (K)"].astype(str) + "_" + c_melt["Acceptor ChainID"].astype(str)
d_melt["Temperature_AcceptorChainID"] = d_melt["Temperature (K)"].astype(str) + "_" + d_melt["Acceptor ChainID"].astype(str)

##### Label amino acid type by colour - for later plotting

red - negatively charged\
blue - positively charged\
green - hydrophobic\
pink - polar\
yellow - special case (cysteine, glycine, proline)

In [ ]:
amino_colours = {"N1":"pink","I2":"green","G3":"yellow","R4":"blue","E5":"red","L6":"green","T7":"pink","D8":"red",\
                "E9":"red","E10": "red","L11": "green","M12":"green","E13": "red","M14":"green","T15":"pink",\
                 "G16":"yellow","G17":"yellow", "S18":"pink","T19":"pink","F20":"green","S21":"pink","I22":"green",\
                 "Q23":"pink"}
def apply_amino_colour(string):
    return str(amino_colours[string])

In [ ]:
amino_type = {"N1":"polar","I2":"hydrophobic","G3":"polar","R4":"positive charge",\
              "E5":"negative charge","L6":"hydrophobic","T7":"polar","D8":"negative charge","E9":"negative charge",\
              "E10": "negative charge","L11": "hydrophobic","M12":"hydrophobic","E13": "negative charge","M14":"hydrophobic",\
              "T15":"polar","G16":"polar","G17":"polar", "S18":"polar","T19":"polar","F20":"hydrophobic",\
              "S21":"polar","I22":"hydrophobic","Q23":"polar"}
def apply_amino_type(string):
    return str(amino_type[string])

In [ ]:
c_melt["Substrate Color Column"] = c_melt["Substrate Joint Position"].apply(apply_amino_colour)
d_melt["Substrate Color Column"] = d_melt["Substrate Joint Position"].apply(apply_amino_colour)

In [ ]:
c_melt["Substrate Type Column"] = c_melt["Substrate Joint Position"].apply(apply_amino_type)
d_melt["Substrate Type Column"] = d_melt["Substrate Joint Position"].apply(apply_amino_type)

In [ ]:
#remove nan values for comparing to lipid interactions/bokeh
c_melt_nonan = c_melt.dropna(inplace = False)
d_melt_nonan = d_melt.dropna(inplace = False)

In [ ]:
# remove small values for plotting
c_melt_remove = c_melt[c_melt["Fraction_Frames"] > 0.05]
c_melt_nonan_remove = c_melt_nonan[c_melt_nonan["Fraction_Frames"] > 0.05]

In [ ]:
# remove small values for plotting
d_melt_remove = d_melt[d_melt["Fraction_Frames"] > 0.05]
d_melt_nonan_remove = d_melt_nonan[d_melt_nonan["Fraction_Frames"] > 0.05]

In [ ]:
#sort values by protein position
c_melt_remove.sort_values("Actual Acceptor Protein ResID", axis = 0, ascending = True, inplace = True) #sort values by protein position
d_melt_remove.sort_values("Actual Acceptor Protein ResID", axis = 0, ascending = True, inplace = True) #sort values by protein position

### Plotting

In [ ]:
palette_dic_chain = {"303.15_A":blue, "310.15_A":orange,"303.15_B":green,"310.15_B":red}

In [ ]:
chain_C_fig = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                          hue='Temperature_AcceptorChainID', data=c_melt_remove, 
                          palette=palette_dic_chain, 
                          hue_order=["303.15_A","310.15_A","303.15_B","310.15_B"],
                          height = 5, aspect = 3).fig; 
plt.xlabel('ResID')
plt.ylabel('Fraction Frames')
chain_C_fig.savefig("Chain C Substrate Interactions by Temp and Chain correct.png", dpi = 300)

In [ ]:
chain_D_fig = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                          hue='Temperature_AcceptorChainID', data=d_melt_remove, 
                          palette=palette_dic_chain, 
                          hue_order=["303.15_A","310.15_A","303.15_B","310.15_B"],
                          height = 5, aspect = 3).fig; 
plt.xlabel('ResID')
plt.ylabel('Fraction Frames')
chain_D_fig.savefig("Chain D Substrate Interactions by Temp and Chain correct.png", dpi = 300)

In [ ]:
palette_dic = {"negative charge":blue, "positive charge":orange,"polar":green,"hydrophobic":red}

In [ ]:
# plot chain C interactions by amino type
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='Substrate Type Column', data=c_melt_remove, 
                               palette=palette_dic,
                               hue_order=["hydrophobic","polar","negative charge","positive charge"],
                               height = 5, aspect = 3).fig; 
plt.xlabel('ResID')
plt.ylabel('Fraction Frames')
chain_C_fig_type.savefig("Chain C Substrate Interactions by amino acid type correct.png", dpi = 300)

In [ ]:
# plot chain D interactions by amino type
chain_D_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='Substrate Type Column', data=d_melt_remove, 
                               palette=palette_dic, 
                               hue_order=["hydrophobic","polar","negative charge","positive charge"],
                               height = 5, aspect = 3).fig; 
plt.xlabel('ResID')
plt.ylabel('Fraction Frames')
chain_D_fig_type.savefig("Chain D Substrate Interactions by amino acid type correct.png", dpi = 300)

### Bokeh Plot

In [ ]:
from bokeh.plotting import Figure, output_notebook, show, save
from bokeh.models import ColumnDataSource, HoverTool, GroupFilter, CDSView
from bokeh.models import BoxSelectTool

output_notebook()

In [ ]:
c_data = ColumnDataSource(c_melt_nonan_remove)

c_int = Figure(tools='pan,wheel_zoom,box_zoom,reset', tooltips=[('Temperature (K)', '@{Temperature (K)}'), ('Chain', '@{Acceptor ChainID}'),('Sub', '@{Substrate Joint Position}'), ('Acceptor', '@{Acceptor Joint Position}')], width=2000, height=300, x_axis_label = "Acceptor Position", y_axis_label = "Fraction Frames")
c_int.scatter(source=c_data, x='Acceptor Protein ResID', y='Fraction_Frames', \
                  fill_color='Substrate Color Column', size=5)
c_int.add_tools(BoxSelectTool(dimensions="width"))

show(c_int);

In [ ]:
d_data = ColumnDataSource(d_melt_nonan_remove)

d_int = Figure(tools='pan,wheel_zoom,box_zoom,reset', tooltips=[('Temperature (K)', '@{Temperature (K)}'), ('Chain', '@{Acceptor ChainID}'),('Sub', '@{Substrate Joint Position}'), ('Acceptor', '@{Acceptor Joint Position}')], width=2000, height=300, x_axis_label = "Acceptor Position", y_axis_label = "Fraction Frames")
d_int.scatter(source=d_data, x='Acceptor Protein ResID', y='Fraction_Frames', \
                  fill_color='Substrate Color Column', size=5)
d_int.add_tools(BoxSelectTool(dimensions="width"))

show(d_int);

##### Save Bokeh Plot

In [ ]:
save(c_int, filename='data/Chain C Substrate Interactions Bokeh', resources='inline', title='Substrate Interactions');

In [ ]:
save(d_int, filename='data/Chain D Substrate Interactions Bokeh', resources='inline', title='Substrate Interactions');

### Timeseries of Specific Protein-Substrate Contacts - Thesis

In [ ]:
# read in data - Q15 Chain A with T19 Sub
u1_C_term_polar_sub = pd.read_csv("u1 Chain A Q15 with Sub T19 Contacts 6 angstrom.csv")
u2_C_term_polar_sub = pd.read_csv("u2 Chain A Q15 with Sub T19 Contacts 6 angstrom.csv")
u3_C_term_polar_sub = pd.read_csv("u3 Chain A Q15 with Sub T19 Contacts 6 angstrom.csv")
u4_C_term_polar_sub = pd.read_csv("u4 Chain A Q15 with Sub T19 Contacts 6 angstrom.csv")

In [ ]:
# read in data - D78 Chain A with R4 Sub
u1_tail_pin_sub = pd.read_csv("u1 Chain A D78 with Sub R4 Contacts 6 angstrom.csv")
u2_tail_pin_sub = pd.read_csv("u2 Chain A D78 with Sub R4 Contacts 6 angstrom.csv")
u3_tail_pin_sub = pd.read_csv("u3 Chain A D78 with Sub R4 Contacts 6 angstrom.csv")
u4_tail_pin_sub = pd.read_csv("u4 Chain A D78 with Sub R4 Contacts 6 angstrom.csv")

In [ ]:
# read in data - K63 K70 K73 Chain A with E5 D8 D10 Sub
u1_middle_pin_sub = pd.read_csv("u1 Chain A K63 K70 K73 with Sub E5 D8 D10 Contacts 6 angstrom.csv")
u2_middle_pin_sub = pd.read_csv("u2 Chain A K63 K70 K73 with Sub E5 D8 D10 Contacts 6 angstrom.csv")
u3_middle_pin_sub = pd.read_csv("u3 Chain A K63 K70 K73 with Sub E5 D8 D10 Contacts 6 angstrom.csv")
u4_middle_pin_sub = pd.read_csv("u4 Chain A K63 K70 K73 with Sub E5 D8 D10 Contacts 6 angstrom.csv")

In [ ]:
# read in data - K499 Chain A with E9 Sub
u1_middle_pin_chainA_nbd_sub = pd.read_csv("u1 Chain A K499 with Sub E9 Contacts 6 angstrom.csv")
u2_middle_pin_chainA_nbd_sub = pd.read_csv("u2 Chain A K499 with Sub E9 Contacts 6 angstrom.csv")
u3_middle_pin_chainA_nbd_sub = pd.read_csv("u3 Chain A K499 with Sub E9 Contacts 6 angstrom.csv")
u4_middle_pin_chainA_nbd_sub = pd.read_csv("u4 Chain A K499 with Sub E9 Contacts 6 angstrom.csv")

In [ ]:
# read in data - K607 Chain B with E13 Sub
u1_middle_pin_chainB_nbd_sub = pd.read_csv("u1 Chain B K607 with Sub E13 Contacts 6 angstrom.csv")
u2_middle_pin_chainB_nbd_sub = pd.read_csv("u2 Chain B K607 with Sub E13 Contacts 6 angstrom.csv")
u3_middle_pin_chainB_nbd_sub = pd.read_csv("u3 Chain B K607 with Sub E13 Contacts 6 angstrom.csv")
u4_middle_pin_chainB_nbd_sub = pd.read_csv("u4 Chain B K607 with Sub E13 Contacts 6 angstrom.csv")

In [ ]:
# read in data - N617 Chain B with S21 Sub
u1_chainB_nbd_sub = pd.read_csv("u2 Chain B N617 with Sub S21 Contacts 6 angstrom.csv")
u2_chainB_nbd_sub = pd.read_csv("u2 Chain B N617 with Sub S21 Contacts 6 angstrom.csv")
u3_chainB_nbd_sub = pd.read_csv("u3 Chain B N617 with Sub S21 Contacts 6 angstrom.csv")
u4_chainB_nbd_sub = pd.read_csv("u4 Chain B N617 with Sub S21 Contacts 6 angstrom.csv")

In [ ]:
# concatonate by temp - Q15 Chain A with T19 Sub
C_term_polar_sub_30_deg = pd.concat([u1_C_term_polar_sub,
                               u2_C_term_polar_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
C_term_polar_sub_37_deg = pd.concat([u3_C_term_polar_sub,
                               u4_C_term_polar_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# concatonate by temp - D78 Chain A with R4 Sub
tail_pin_sub_30_deg = pd.concat([u1_tail_pin_sub,
                               u2_tail_pin_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
tail_pin_sub_37_deg = pd.concat([u3_tail_pin_sub,
                               u4_tail_pin_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# concatonate by temp - K63 K70 K73 Chain A with E5 D8 D10 Sub
middle_pin_sub_30_deg = pd.concat([u1_middle_pin_sub,
                               u2_middle_pin_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
middle_pin_sub_37_deg = pd.concat([u3_middle_pin_sub,
                               u4_middle_pin_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# concatonate by temp - K499 Chain A with E9 Sub
middle_pin_chainA_nbd_sub_30_deg = pd.concat([u1_middle_pin_chainA_nbd_sub,
                               u2_middle_pin_chainA_nbd_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
middle_pin_chainA_nbd_sub_37_deg = pd.concat([u3_middle_pin_chainA_nbd_sub,
                               u4_middle_pin_chainA_nbd_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# concatonate by temp - K607 Chain B with E13 Sub
middle_pin_chainB_nbd_sub_30_deg = pd.concat([u1_middle_pin_chainB_nbd_sub,
                               u2_middle_pin_chainB_nbd_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
middle_pin_chainB_nbd_sub_37_deg = pd.concat([u3_middle_pin_chainB_nbd_sub,
                               u4_middle_pin_chainB_nbd_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# concatonate by temp - N617 Chain B with S21 Sub
chainB_nbd_sub_30_deg = pd.concat([u1_chainB_nbd_sub,
                               u2_chainB_nbd_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
chainB_nbd_sub_37_deg = pd.concat([u3_chainB_nbd_sub,
                               u4_chainB_nbd_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# Reset index - Q15 Chain A with T19 Sub
C_term_polar_sub_30_deg.reset_index(inplace = True)
C_term_polar_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Reset index - D78 Chain A with R4 Sub
tail_pin_sub_30_deg.reset_index(inplace = True)
tail_pin_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Reset index - K63 K70 K73 Chain A with E5 D8 D10 Sub
middle_pin_sub_30_deg.reset_index(inplace = True)
middle_pin_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Reset index - K499 Chain A with E9 Sub
middle_pin_chainA_nbd_sub_30_deg.reset_index(inplace = True)
middle_pin_chainA_nbd_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Reset index - K607 Chain B with E13 Sub
middle_pin_chainB_nbd_sub_30_deg.reset_index(inplace = True)
middle_pin_chainB_nbd_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Reset index - N617 Chain B with S21 Sub
chainB_nbd_sub_30_deg.reset_index(inplace = True)
chainB_nbd_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Add in rolling averages - Q15 Chain A with T19 Sub
C_term_polar_sub_30_deg_rolling = C_term_polar_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
C_term_polar_sub_37_deg_rolling = C_term_polar_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Add in rolling averages - D78 Chain A with R4 Sub
tail_pin_sub_30_deg_rolling = tail_pin_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
tail_pin_sub_37_deg_rolling = tail_pin_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Add in rolling averages - K63 K70 K73 Chain A with E5 D8 D10 Sub
middle_pin_sub_30_deg_rolling = middle_pin_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
middle_pin_sub_37_deg_rolling = middle_pin_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Add in rolling averages - K499 Chain A with E9 Sub
middle_pin_chainA_nbd_sub_30_deg_rolling = middle_pin_chainA_nbd_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
middle_pin_chainA_nbd_sub_37_deg_rolling = middle_pin_chainA_nbd_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Add in rolling averages - K607 Chain B with E13 Sub
middle_pin_chainB_nbd_sub_30_deg_rolling = middle_pin_chainB_nbd_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
middle_pin_chainB_nbd_sub_37_deg_rolling = middle_pin_chainB_nbd_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Add in rolling averages - N617 Chain B with S21 Sub
chainB_nbd_sub_30_deg_rolling = chainB_nbd_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
chainB_nbd_sub_37_deg_rolling = chainB_nbd_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Plot with rolling averages - Q15 Chain A with T19 Sub
#plt.errorbar(x=C_term_polar_sub_30_deg["Time (ns)"],y=C_term_polar_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=C_term_polar_sub_30_deg["Time (ns)"],y=C_term_polar_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=C_term_polar_sub_37_deg["Time (ns)"],y=C_term_polar_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=C_term_polar_sub_37_deg["Time (ns)"],y=C_term_polar_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain A Q15 with Sub T19")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain A Q15 with Sub T19.png", dpi=300)

In [ ]:
# Plot with rolling averages - D78 Chain A with R4 Sub
#plt.errorbar(x=tail_pin_sub_30_deg["Time (ns)"],y=tail_pin_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=tail_pin_sub_30_deg["Time (ns)"],y=tail_pin_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=tail_pin_sub_37_deg["Time (ns)"],y=tail_pin_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=tail_pin_sub_37_deg["Time (ns)"],y=tail_pin_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain A D78 with Sub R4")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain A D78 with Sub R4.png", dpi=300)

In [ ]:
# Plot with rolling averages - K63 K70 K73 Chain A with E5 D8 D10 Sub
#plt.errorbar(x=middle_pin_sub_30_deg["Time (ns)"],y=middle_pin_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=middle_pin_sub_30_deg["Time (ns)"],y=middle_pin_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=middle_pin_sub_37_deg["Time (ns)"],y=middle_pin_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=middle_pin_sub_37_deg["Time (ns)"],y=middle_pin_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain A K63 K70 K73 with Sub E5 D8 D10")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain A K63 K70 K73 with Sub E5 D8 D10.png", dpi=300)

In [ ]:
# Plot with rolling averages - K499 Chain A with E9 Sub
#plt.errorbar(x=middle_pin_chainA_nbd_sub_30_deg["Time (ns)"],y=middle_pin_chainA_nbd_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=middle_pin_chainA_nbd_sub_30_deg["Time (ns)"],y=middle_pin_chainA_nbd_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=middle_pin_chainA_nbd_sub_37_deg["Time (ns)"],y=middle_pin_chainA_nbd_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=middle_pin_chainA_nbd_sub_37_deg["Time (ns)"],y=middle_pin_chainA_nbd_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain A K499 with Sub E9")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain A K499 with Sub E9.png", dpi=300)

In [ ]:
# Plot with rolling averages - K607 Chain B with E13 Sub
#plt.errorbar(x=middle_pin_chainB_nbd_sub_30_deg["Time (ns)"],y=middle_pin_chainB_nbd_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=middle_pin_chainB_nbd_sub_30_deg["Time (ns)"],y=middle_pin_chainB_nbd_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=middle_pin_chainB_nbd_sub_37_deg["Time (ns)"],y=middle_pin_chainB_nbd_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=middle_pin_chainB_nbd_sub_37_deg["Time (ns)"],y=middle_pin_chainB_nbd_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain B K607 with Sub E13")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain B K607 with Sub E13.png", dpi=300)

In [ ]:
# Plot with rolling averages - N617 Chain B with S21 Sub
#plt.errorbar(x=chainB_nbd_sub_30_deg["Time (ns)"],y=chainB_nbd_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=chainB_nbd_sub_30_deg["Time (ns)"],y=chainB_nbd_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=chainB_nbd_sub_37_deg["Time (ns)"],y=chainB_nbd_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=chainB_nbd_sub_37_deg["Time (ns)"],y=chainB_nbd_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain B N617 with Sub S21")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain B N617 with Sub S21.png", dpi=300)

### Membrane Enrichment

Look at using radial distribution curve for each lipid - CDL2, POPE, and POPG and see if it tells us about how these lipids interact in general with target amino acids. Function looks at density of lipids around a target reference.

In [ ]:
# Reimport all contacts for plotting - with substrate
cdl2_sub_30_deg_traj_1 = pd.read_csv("data/CDL2_Band_Enrichment_Sub_30_Deg_Traj_1.csv")
cdl2_sub_37_deg_traj_1 = pd.read_csv("data/CDL2_Band_Enrichment_Sub_37_Deg_Traj_1.csv")
cdl2_sub_30_deg_traj_2 = pd.read_csv("data/CDL2_Band_Enrichment_Sub_30_Deg_Traj_2.csv")
cdl2_sub_37_deg_traj_2 = pd.read_csv("data/CDL2_Band_Enrichment_Sub_37_Deg_Traj_2.csv")
pope_sub_30_deg_traj_1 = pd.read_csv("data/POPE_Band_Enrichment_Sub_30_Deg_Traj_1.csv")
pope_sub_37_deg_traj_1 = pd.read_csv("data/POPE_Band_Enrichment_Sub_37_Deg_Traj_1.csv")
pope_sub_30_deg_traj_2 = pd.read_csv("data/POPE_Band_Enrichment_Sub_30_Deg_Traj_2.csv")
pope_sub_37_deg_traj_2 = pd.read_csv("data/POPE_Band_Enrichment_Sub_37_Deg_Traj_2.csv")
popg_sub_30_deg_traj_1 = pd.read_csv("data/POPG_Band_Enrichment_Sub_30_Deg_Traj_1.csv")
popg_sub_37_deg_traj_1 = pd.read_csv("data/POPG_Band_Enrichment_Sub_37_Deg_Traj_1.csv")
popg_sub_30_deg_traj_2 = pd.read_csv("data/POPG_Band_Enrichment_Sub_30_Deg_Traj_2.csv")
popg_sub_37_deg_traj_2 = pd.read_csv("data/POPG_Band_Enrichment_Sub_37_Deg_Traj_2.csv")

In [ ]:
# Reimport all contacts for plotting - without substrate
cdl2_nosub_30_deg_traj_1 = pd.read_csv("data/CDL2_Band_Enrichment_Nosub_30_Deg_Traj_1.csv")
cdl2_nosub_37_deg_traj_1 = pd.read_csv("data/CDL2_Band_Enrichment_Nosub_37_Deg_Traj_1.csv")
cdl2_nosub_30_deg_traj_2 = pd.read_csv("data/CDL2_Band_Enrichment_Nosub_30_Deg_Traj_2.csv")
cdl2_nosub_37_deg_traj_2 = pd.read_csv("data/CDL2_Band_Enrichment_Nosub_37_Deg_Traj_2.csv")
pope_nosub_30_deg_traj_1 = pd.read_csv("data/POPE_Band_Enrichment_Nosub_30_Deg_Traj_1.csv")
pope_nosub_37_deg_traj_1 = pd.read_csv("data/POPE_Band_Enrichment_Nosub_37_Deg_Traj_1.csv")
pope_nosub_30_deg_traj_2 = pd.read_csv("data/POPE_Band_Enrichment_Nosub_30_Deg_Traj_2.csv")
pope_nosub_37_deg_traj_2 = pd.read_csv("data/POPE_Band_Enrichment_Nosub_37_Deg_Traj_2.csv")
popg_nosub_30_deg_traj_1 = pd.read_csv("data/POPG_Band_Enrichment_Nosub_30_Deg_Traj_1.csv")
popg_nosub_37_deg_traj_1 = pd.read_csv("data/POPG_Band_Enrichment_Nosub_37_Deg_Traj_1.csv")
popg_nosub_30_deg_traj_2 = pd.read_csv("data/POPG_Band_Enrichment_Nosub_30_Deg_Traj_2.csv")
popg_nosub_37_deg_traj_2 = pd.read_csv("data/POPG_Band_Enrichment_Nosub_37_Deg_Traj_2.csv")

In [ ]:
# Concat trajectory repeats - substrate
cdl2_sub_30_deg = pd.concat([cdl2_sub_30_deg_traj_1,cdl2_sub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
cdl2_sub_37_deg = pd.concat([cdl2_sub_37_deg_traj_1,cdl2_sub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
pope_sub_30_deg = pd.concat([pope_sub_30_deg_traj_1,pope_sub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
pope_sub_37_deg = pd.concat([pope_sub_37_deg_traj_1,pope_sub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
popg_sub_30_deg = pd.concat([popg_sub_30_deg_traj_1,popg_sub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
popg_sub_37_deg = pd.concat([popg_sub_37_deg_traj_1,popg_sub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))

In [ ]:
# Concat trajectory repeats - no substrate
cdl2_nosub_30_deg = pd.concat([cdl2_nosub_30_deg_traj_1,cdl2_nosub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
cdl2_nosub_37_deg = pd.concat([cdl2_nosub_37_deg_traj_1,cdl2_nosub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
pope_nosub_30_deg = pd.concat([pope_nosub_30_deg_traj_1,pope_nosub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
pope_nosub_37_deg = pd.concat([pope_nosub_37_deg_traj_1,pope_nosub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
popg_nosub_30_deg = pd.concat([popg_nosub_30_deg_traj_1,popg_nosub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
popg_nosub_37_deg = pd.concat([popg_nosub_37_deg_traj_1,popg_nosub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))

In [ ]:
#Reset index to remove multi-indexing - substrate
cdl2_sub_30_deg.reset_index(inplace = True)
cdl2_sub_37_deg.reset_index(inplace = True)
pope_sub_30_deg.reset_index(inplace = True) 
pope_sub_37_deg.reset_index(inplace = True) 
popg_sub_30_deg.reset_index(inplace = True) 
popg_sub_37_deg.reset_index(inplace = True) 

In [ ]:
#Reset index to remove multi-indexing - no substrate
cdl2_nosub_30_deg.reset_index(inplace = True)
cdl2_nosub_37_deg.reset_index(inplace = True)
pope_nosub_30_deg.reset_index(inplace = True) 
pope_nosub_37_deg.reset_index(inplace = True) 
popg_nosub_30_deg.reset_index(inplace = True) 
popg_nosub_37_deg.reset_index(inplace = True)

In [ ]:
# Plot by temp for substrate
plt.errorbar(x=cdl2_sub_30_deg["Band Distance"],y=cdl2_sub_30_deg["avg"],yerr=cdl2_sub_30_deg["std"],xerr=2.5,label="CDL2 303.15 K")
plt.errorbar(x=pope_sub_30_deg["Band Distance"],y=pope_sub_30_deg["avg"],yerr=pope_sub_30_deg["std"],xerr=2.5,label="POPE 303.15 K")
plt.errorbar(x=popg_sub_30_deg["Band Distance"],y=popg_sub_30_deg["avg"],yerr=popg_sub_30_deg["std"],xerr=2.5,label="POPG 303.15 K")
plt.errorbar(x=cdl2_sub_37_deg["Band Distance"],y=cdl2_sub_37_deg["avg"],yerr=cdl2_sub_37_deg["std"],xerr=2.5,label="CDL2 310.15 K")
plt.errorbar(x=pope_sub_37_deg["Band Distance"],y=pope_sub_37_deg["avg"],yerr=pope_sub_37_deg["std"],xerr=2.5,label="POPE 310.15 K")
plt.errorbar(x=popg_sub_37_deg["Band Distance"],y=popg_sub_37_deg["avg"],yerr=popg_sub_37_deg["std"],xerr=2.5,label="POPG 310.15 K")
plt.legend(loc='best')
plt.title("Lipid Enrichment with Substrate")
plt.xlabel("Radial Distance From Protein (Angstrom)")
plt.ylabel("Enrichment")
plt.savefig("Lipid Enrichment by Temp with Substrate", dpi=300)

In [ ]:
# Plot by temp no substrate
plt.errorbar(x=cdl2_nosub_30_deg["Band Distance"],y=cdl2_nosub_30_deg["avg"],yerr=cdl2_nosub_30_deg["std"],xerr=2.5,label="CDL2 303.15 K")
plt.errorbar(x=pope_nosub_30_deg["Band Distance"],y=pope_nosub_30_deg["avg"],yerr=pope_nosub_30_deg["std"],xerr=2.5,label="POPE 303.15 K")
plt.errorbar(x=popg_nosub_30_deg["Band Distance"],y=popg_nosub_30_deg["avg"],yerr=popg_nosub_30_deg["std"],xerr=2.5,label="POPG 303.15 K")
plt.errorbar(x=cdl2_nosub_37_deg["Band Distance"],y=cdl2_nosub_37_deg["avg"],yerr=cdl2_nosub_37_deg["std"],xerr=2.5,label="CDL2 310.15 K")
plt.errorbar(x=pope_nosub_37_deg["Band Distance"],y=pope_nosub_37_deg["avg"],yerr=pope_nosub_37_deg["std"],xerr=2.5,label="POPE 310.15 K")
plt.errorbar(x=popg_nosub_37_deg["Band Distance"],y=popg_nosub_37_deg["avg"],yerr=popg_nosub_37_deg["std"],xerr=2.5,label="POPG 310.15 K")
plt.legend(loc='best')
plt.title("Lipid Enrichment without Substrate")
plt.xlabel("Radial Distance From Protein (Angstrom)")
plt.ylabel("Enrichment")
plt.savefig("Lipid Enrichment by Temp without Substrate", dpi=300)

In [ ]:
# Plot sub vs no sub 303.15 K
plt.errorbar(x=cdl2_sub_30_deg["Band Distance"],y=cdl2_sub_30_deg["avg"],yerr=cdl2_sub_30_deg["std"],xerr=2.5,label="CDL2 Substrate")
plt.errorbar(x=cdl2_nosub_30_deg["Band Distance"],y=cdl2_nosub_30_deg["avg"],yerr=cdl2_nosub_30_deg["std"],xerr=2.5,label="CDL2 No Substrate")
plt.errorbar(x=pope_sub_30_deg["Band Distance"],y=pope_sub_30_deg["avg"],yerr=pope_sub_30_deg["std"],xerr=2.5,label="POPE Substrate")
plt.errorbar(x=pope_nosub_30_deg["Band Distance"],y=pope_nosub_30_deg["avg"],yerr=pope_nosub_30_deg["std"],xerr=2.5,label="POPE No Substrate")
plt.errorbar(x=popg_sub_30_deg["Band Distance"],y=popg_sub_30_deg["avg"],yerr=popg_sub_30_deg["std"],xerr=2.5,label="POPG Substrate")
plt.errorbar(x=popg_nosub_30_deg["Band Distance"],y=popg_nosub_30_deg["avg"],yerr=popg_nosub_30_deg["std"],xerr=2.5,label="POPG No Substrate")
plt.legend(loc='best')
plt.title("Lipid Enrichment Substrate vs No Substrate at 303.15 K")
plt.xlabel("Radial Distance From Protein (Angstrom)")
plt.ylabel("Enrichment")
plt.savefig("Lipid Enrichment Substrate vs No Substrate 30 Deg", dpi=300)

In [ ]:
# Plot sub vs no sub 310.15 K
plt.errorbar(x=cdl2_sub_37_deg["Band Distance"],y=cdl2_sub_37_deg["avg"],yerr=cdl2_sub_37_deg["std"],xerr=2.5,label="CDL2 Substrate")
plt.errorbar(x=cdl2_nosub_37_deg["Band Distance"],y=cdl2_nosub_37_deg["avg"],yerr=cdl2_nosub_37_deg["std"],xerr=2.5,label="CDL2 No Substrate")
plt.errorbar(x=pope_sub_37_deg["Band Distance"],y=pope_sub_37_deg["avg"],yerr=pope_sub_37_deg["std"],xerr=2.5,label="POPE Substrate")
plt.errorbar(x=pope_nosub_37_deg["Band Distance"],y=pope_nosub_37_deg["avg"],yerr=pope_nosub_37_deg["std"],xerr=2.5,label="POPE No Substrate")
plt.errorbar(x=popg_sub_37_deg["Band Distance"],y=popg_sub_37_deg["avg"],yerr=popg_sub_37_deg["std"],xerr=2.5,label="POPG Substrate")
plt.errorbar(x=popg_nosub_37_deg["Band Distance"],y=popg_nosub_37_deg["avg"],yerr=popg_nosub_37_deg["std"],xerr=2.5,label="POPG No Substrate")
plt.legend(loc=9)
plt.title("Lipid Enrichment Substrate vs No Substrate at 310.15 K")
plt.xlabel("Radial Distance From Protein (Angstrom)")
plt.ylabel("Enrichment")
plt.savefig("Lipid Enrichment Substrate vs No Substrate 37 Deg", dpi=300)

### Lipid Clustering Analysis

Investigate lipid clustering in different domains of 6v9z from substrate vs no substrate trajectories

In [ ]:
# Data import with substrate - by domain only
lipid_clust_sub_30_deg_traj_1 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_sub_30_deg_traj_1.csv")
lipid_clust_sub_30_deg_traj_2 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_sub_30_deg_traj_2.csv")
lipid_clust_sub_37_deg_traj_1 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_sub_37_deg_traj_1.csv")
lipid_clust_sub_37_deg_traj_2 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_sub_37_deg_traj_2.csv")

In [ ]:
# Data import without substrate - by domain only
lipid_clust_nosub_30_deg_traj_1 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_nosub_30_deg_traj_1.csv")
lipid_clust_nosub_30_deg_traj_2 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_nosub_30_deg_traj_2.csv")
lipid_clust_nosub_37_deg_traj_1 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_nosub_37_deg_traj_1.csv")
lipid_clust_nosub_37_deg_traj_2 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_nosub_37_deg_traj_2.csv")

In [ ]:
# Data import with substrate - by chain and domain
lipid_clust_sub_30_deg_traj_1_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_sub_30_deg_traj_1.csv")
lipid_clust_sub_30_deg_traj_2_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_sub_30_deg_traj_2.csv")
lipid_clust_sub_37_deg_traj_1_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_sub_37_deg_traj_1.csv")
lipid_clust_sub_37_deg_traj_2_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_sub_37_deg_traj_2.csv")

In [ ]:
# Data import without substrate - by chain and domain
lipid_clust_nosub_30_deg_traj_1_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_nosub_30_deg_traj_1.csv")
lipid_clust_nosub_30_deg_traj_2_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_nosub_30_deg_traj_2.csv")
lipid_clust_nosub_37_deg_traj_1_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_nosub_37_deg_traj_1.csv")
lipid_clust_nosub_37_deg_traj_2_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_nosub_37_deg_traj_2.csv")

In [ ]:
# Combine temps as well & concat
lipid_clust_sub = pd.concat([lipid_clust_sub_30_deg_traj_1,lipid_clust_sub_30_deg_traj_2,lipid_clust_sub_37_deg_traj_1,lipid_clust_sub_37_deg_traj_2],ignore_index=True)
lipid_clust_nosub = pd.concat([lipid_clust_nosub_30_deg_traj_1,lipid_clust_nosub_30_deg_traj_2,lipid_clust_nosub_37_deg_traj_1,lipid_clust_nosub_37_deg_traj_2],ignore_index=True)

In [ ]:
# Combine temps as well & concat - by chain
lipid_clust_sub_bychain = pd.concat([lipid_clust_sub_30_deg_traj_1_bychain,lipid_clust_sub_30_deg_traj_2_bychain,lipid_clust_sub_37_deg_traj_1_bychain,lipid_clust_sub_37_deg_traj_2_bychain],ignore_index=True)
lipid_clust_nosub_bychain = pd.concat([lipid_clust_nosub_30_deg_traj_1_bychain,lipid_clust_nosub_30_deg_traj_2_bychain,lipid_clust_nosub_37_deg_traj_1_bychain,lipid_clust_nosub_37_deg_traj_2_bychain],ignore_index=True)

In [ ]:
# Reset index to remove multi-indexing for combined temps
lipid_clust_sub.reset_index(inplace = True)
lipid_clust_nosub.reset_index(inplace = True)

In [ ]:
# Reset index to remove multi-indexing for combined temps - by domain & chain
lipid_clust_sub_bychain.reset_index(inplace = True)
lipid_clust_nosub_bychain.reset_index(inplace = True)

In [ ]:
# Add in substrate info
lipid_clust_sub["Substrate"] = "yes"
lipid_clust_nosub["Substrate"] = "no"

In [ ]:
# Add in substrate info - by domain & chain
lipid_clust_sub_bychain["Substrate"] = "yes"
lipid_clust_nosub_bychain["Substrate"] = "no"

In [ ]:
# Concat sub and no sub for plotting
lipid_clust = pd.concat([lipid_clust_sub,lipid_clust_nosub])

In [ ]:
# Concat sub and no sub for plotting - by domain & chain
lipid_clust_bychain = pd.concat([lipid_clust_sub_bychain,lipid_clust_nosub_bychain])

In [ ]:
# Add label for seaborn plot
lipid_clust["Label"] = lipid_clust["Substrate"].astype(str) + " " + lipid_clust["Lipid"].astype(str)

In [ ]:
# Add label for seaborn plot - by domain & chain
lipid_clust_bychain["Label"] = lipid_clust_bychain["Substrate"].astype(str) + " " + lipid_clust_bychain["Lipid"].astype(str)

#### Plots

In [ ]:
fig_dims = (30, 10)
fig, ax = plt.subplots(figsize=fig_dims)
clust_fig_label = sns.barplot(data=lipid_clust, x='Domain', y='Scaled_Median_Count', hue='Label',estimator=np.mean,ci=90).get_figure();
clust_fig_label.savefig("Lipid Clustering", dpi=300)

In [ ]:
fig_dims = (30, 10)
fig, ax = plt.subplots(figsize=fig_dims)
clust_fig = sns.barplot(data=lipid_clust, x='Domain', y='Scaled_Median_Count', hue='Substrate',estimator=np.mean,ci=90).get_figure();
clust_fig.savefig("Lipid Clustering by substrate", dpi=300)

In [ ]:
fig_dims = (30, 10)
fig, ax = plt.subplots(figsize=fig_dims)
clust_fig_bychain = sns.barplot(data=lipid_clust_bychain, x='Domain', y='Scaled_Median_Count', hue='Substrate',estimator=np.mean,ci=90).get_figure();
clust_fig_bychain.savefig("Lipid Clustering by substrate and chain", dpi=300)

In [ ]:
fig_dims = (30, 10)
fig, ax = plt.subplots(figsize=fig_dims)
clust_fig_bychain_label = sns.barplot(data=lipid_clust_bychain, x='Domain', y='Scaled_Median_Count', hue='Label',estimator=np.mean,ci=90).get_figure();
clust_fig_bychain_label.savefig("Lipid Clustering by chain", dpi=300)

### Permutation Test - Lipid Clustering

Test permutation test against my data & see differences in the output distributions.

In [ ]:
# Extract values for the test separated by substrate and no substrate
perm_sub_clust, perm_nosub_clust = extract_sub_vals_clust(lipid_clust_bychain)

In [ ]:
#Get permutation test distribution for substrate vs no substrate lipid clustering
pooled = np.hstack([perm_sub_clust,perm_nosub_clust])
sizeA = 95
sizeB= 95
iterations = 1000000
test = permutation(pooled,sizeA,sizeB,iterations)

In [ ]:
#Plot permutation test histogram
plt.hist(test,bins=100);
plt.title("Permutation Test Substrate vs No Substrate")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Permutation Test Sub vs No Sub Lipid Clustering", dpi=150)

In [ ]:
#Calculate p-values using permutation test distribution
p_val_clust_perm = p_val(diff_lipid_clust,iterations,test)

In [ ]:
#Convert p-values to series for saving
p_val_clust_perm_series = pd.Series(p_val_clust_perm, name='uncorrected p-value')
p_val_clust_perm_series.index.name = "Domain"
p_val_clust_perm_series.reset_index()

In [ ]:
#save p-values!
p_val_clust_perm_series.to_csv('Lipid clustering permutation p-values.csv')

In [ ]:
#Filter for significant p-values for CDL2 preference
perm_clust_p_val_sig = sig_p_val(p_val_clust_perm,0.05)

5 regions which show significant differences (1% sig level) when substrate is present & not present based on the permutation test (p-vals are corrected):

Chain A Transmembrane Top: CDL2 --> p-val 0\
Chain A Transmembrane Top: POPE --> p-val 0\
Chain B Substrate Entrance Channel: POPE --> p-val 0.000003\
Chain B Transmembrane Top: CDL2 --> p-val 0\
Chain B Transmembrane Top: POPE --> p-val 0

7 regions which show significant differences (5% sig level) when substrate is present & not present based on the permutation test (p-vals are corrected):

Chain A Transmembrane Arm: CDL2 --> p-val 0.00143\
Chain A Transmembrane Top: CDL2 --> p-val 0\
Chain A Transmembrane Top: POPE --> p-val 0\
Chain A Transmembrane Top: POPG --> p-val 0.001079\
Chain B Substrate Entrance Channel: POPE --> p-val 0.000003\
Chain B Transmembrane Top: CDL2 --> p-val 0\
Chain B Transmembrane Top: POPE --> p-val 0

These lipids are depleted when the substrate is present.

## SASA Analysis

In [ ]:
# Chain A 30 degrees
chain_A_traj_1_sub_30_deg_x,chain_A_traj_1_sub_30_deg_y = np.loadtxt("data/Traj_1_Sub_30_deg_Chain_A_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_2_sub_30_deg_x,chain_A_traj_2_sub_30_deg_y = np.loadtxt("data/Traj_2_Sub_30_deg_Chain_A_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_1_nosub_30_deg_x,chain_A_traj_1_nosub_30_deg_y = np.loadtxt("data/Traj_1_NoSub_30_deg_Chain_A_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_2_nosub_30_deg_x,chain_A_traj_2_nosub_30_deg_y = np.loadtxt("data/Traj_2_NoSub_30_deg_Chain_A_PEP_sasa.xvg",comments=["@","#"],unpack=True)

In [ ]:
# Chain B 30 degrees
chain_B_traj_1_sub_30_deg_x,chain_B_traj_1_sub_30_deg_y = np.loadtxt("data/Traj_1_Sub_30_deg_Chain_B_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_2_sub_30_deg_x,chain_B_traj_2_sub_30_deg_y = np.loadtxt("data/Traj_2_Sub_30_deg_Chain_B_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_1_nosub_30_deg_x,chain_B_traj_1_nosub_30_deg_y = np.loadtxt("data/Traj_1_NoSub_30_deg_Chain_B_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_2_nosub_30_deg_x,chain_B_traj_2_nosub_30_deg_y = np.loadtxt("data/Traj_2_NoSub_30_deg_Chain_B_PEP_sasa.xvg",comments=["@","#"],unpack=True)

In [ ]:
# Chain A 37 degrees
chain_A_traj_1_sub_37_deg_x,chain_A_traj_1_sub_37_deg_y = np.loadtxt("data/Traj_1_Sub_37_deg_Chain_A_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_2_sub_37_deg_x,chain_A_traj_2_sub_37_deg_y = np.loadtxt("data/Traj_2_Sub_37_deg_Chain_A_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_1_nosub_37_deg_x,chain_A_traj_1_nosub_37_deg_y = np.loadtxt("data/Traj_1_NoSub_37_deg_Chain_A_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_2_nosub_37_deg_x,chain_A_traj_2_nosub_37_deg_y = np.loadtxt("data/Traj_2_NoSub_37_deg_Chain_A_PEP_sasa.xvg",comments=["@","#"],unpack=True)

In [ ]:
# Chain B 37 degrees
chain_B_traj_1_sub_37_deg_x,chain_B_traj_1_sub_37_deg_y = np.loadtxt("data/Traj_1_Sub_37_deg_Chain_B_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_2_sub_37_deg_x,chain_B_traj_2_sub_37_deg_y = np.loadtxt("data/Traj_2_Sub_37_deg_Chain_B_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_1_nosub_37_deg_x,chain_B_traj_1_nosub_37_deg_y = np.loadtxt("data/Traj_1_NoSub_37_deg_Chain_B_PEP_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_2_nosub_37_deg_x,chain_B_traj_2_nosub_37_deg_y = np.loadtxt("data/Traj_2_NoSub_37_deg_Chain_B_PEP_sasa.xvg",comments=["@","#"],unpack=True)

In [ ]:
# Chain A 30 degrees data dicts
d_traj_1_chain_A_sub_30_deg = {"Time(ns)":chain_A_traj_1_sub_30_deg_x,'Area':chain_A_traj_1_sub_30_deg_y}
d_traj_2_chain_A_sub_30_deg = {"Time(ns)":chain_A_traj_1_sub_30_deg_x,'Area':chain_A_traj_1_sub_30_deg_y}
d_traj_1_chain_A_nosub_30_deg = {"Time(ns)":chain_A_traj_1_nosub_30_deg_x,'Area':chain_A_traj_1_nosub_30_deg_y}
d_traj_2_chain_A_nosub_30_deg = {"Time(ns)":chain_A_traj_1_nosub_30_deg_x,'Area':chain_A_traj_1_nosub_30_deg_y}

In [ ]:
# Chain B 30 degrees data dicts
d_traj_1_chain_B_sub_30_deg = {"Time(ns)":chain_B_traj_1_sub_30_deg_x,'Area':chain_B_traj_1_sub_30_deg_y}
d_traj_2_chain_B_sub_30_deg = {"Time(ns)":chain_B_traj_1_sub_30_deg_x,'Area':chain_B_traj_1_sub_30_deg_y}
d_traj_1_chain_B_nosub_30_deg = {"Time(ns)":chain_B_traj_1_nosub_30_deg_x,'Area':chain_B_traj_1_nosub_30_deg_y}
d_traj_2_chain_B_nosub_30_deg = {"Time(ns)":chain_B_traj_1_nosub_30_deg_x,'Area':chain_B_traj_1_nosub_30_deg_y}

In [ ]:
# Chain A 37 degrees data dicts
d_traj_1_chain_A_sub_37_deg = {"Time(ns)":chain_A_traj_1_sub_37_deg_x,'Area':chain_A_traj_1_sub_37_deg_y}
d_traj_2_chain_A_sub_37_deg = {"Time(ns)":chain_A_traj_1_sub_37_deg_x,'Area':chain_A_traj_1_sub_37_deg_y}
d_traj_1_chain_A_nosub_37_deg = {"Time(ns)":chain_A_traj_1_nosub_37_deg_x,'Area':chain_A_traj_1_nosub_37_deg_y}
d_traj_2_chain_A_nosub_37_deg = {"Time(ns)":chain_A_traj_1_nosub_37_deg_x,'Area':chain_A_traj_1_nosub_37_deg_y}

In [ ]:
# Chain B 37 degrees data dicts
d_traj_1_chain_B_sub_37_deg = {"Time(ns)":chain_B_traj_1_sub_37_deg_x,'Area':chain_B_traj_1_sub_37_deg_y}
d_traj_2_chain_B_sub_37_deg = {"Time(ns)":chain_B_traj_1_sub_37_deg_x,'Area':chain_B_traj_1_sub_37_deg_y}
d_traj_1_chain_B_nosub_37_deg = {"Time(ns)":chain_B_traj_1_nosub_37_deg_x,'Area':chain_B_traj_1_nosub_37_deg_y}
d_traj_2_chain_B_nosub_37_deg = {"Time(ns)":chain_B_traj_1_nosub_37_deg_x,'Area':chain_B_traj_1_nosub_37_deg_y}

In [ ]:
# Chain A 30 degrees dataframes from dicts
traj_1_chain_A_sub_30_deg = pd.DataFrame(d_traj_1_chain_A_sub_30_deg)
traj_2_chain_A_sub_30_deg = pd.DataFrame(d_traj_2_chain_A_sub_30_deg)
traj_1_chain_A_nosub_30_deg = pd.DataFrame(d_traj_1_chain_A_nosub_30_deg)
traj_2_chain_A_nosub_30_deg = pd.DataFrame(d_traj_2_chain_A_nosub_30_deg)

In [ ]:
# Chain B 30 degrees dataframes from dicts
traj_1_chain_B_sub_30_deg = pd.DataFrame(d_traj_1_chain_B_sub_30_deg)
traj_2_chain_B_sub_30_deg = pd.DataFrame(d_traj_2_chain_B_sub_30_deg)
traj_1_chain_B_nosub_30_deg = pd.DataFrame(d_traj_1_chain_B_nosub_30_deg)
traj_2_chain_B_nosub_30_deg = pd.DataFrame(d_traj_2_chain_B_nosub_30_deg)

In [ ]:
# Chain A 37 degrees dataframes from dicts
traj_1_chain_A_sub_37_deg = pd.DataFrame(d_traj_1_chain_A_sub_37_deg)
traj_2_chain_A_sub_37_deg = pd.DataFrame(d_traj_2_chain_A_sub_37_deg)
traj_1_chain_A_nosub_37_deg = pd.DataFrame(d_traj_1_chain_A_nosub_37_deg)
traj_2_chain_A_nosub_37_deg = pd.DataFrame(d_traj_2_chain_A_nosub_37_deg)

In [ ]:
# Chain B 37 degrees dataframes from dicts
traj_1_chain_B_sub_37_deg = pd.DataFrame(d_traj_1_chain_B_sub_37_deg)
traj_2_chain_B_sub_37_deg = pd.DataFrame(d_traj_2_chain_B_sub_37_deg)
traj_1_chain_B_nosub_37_deg = pd.DataFrame(d_traj_1_chain_B_nosub_37_deg)
traj_2_chain_B_nosub_37_deg = pd.DataFrame(d_traj_2_chain_B_nosub_37_deg)

In [ ]:
# Chain A 30 degrees aggregate by trajectory
chain_A_sub_30_deg = pd.concat([traj_1_chain_A_sub_30_deg,
                               traj_2_chain_A_sub_30_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))
chain_A_nosub_30_deg = pd.concat([traj_1_chain_A_nosub_30_deg,
                               traj_2_chain_A_nosub_30_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))

In [ ]:
# Chain B 30 degrees aggregate by trajectory
chain_B_sub_30_deg = pd.concat([traj_1_chain_B_sub_30_deg,
                               traj_2_chain_B_sub_30_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))
chain_B_nosub_30_deg = pd.concat([traj_1_chain_B_nosub_30_deg,
                               traj_2_chain_B_nosub_30_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))

In [ ]:
# Chain A 37 degrees aggregate by trajectory
chain_A_sub_37_deg = pd.concat([traj_1_chain_A_sub_37_deg,
                               traj_2_chain_A_sub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))
chain_A_nosub_37_deg = pd.concat([traj_1_chain_A_nosub_37_deg,
                               traj_2_chain_A_nosub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))

In [ ]:
# Chain B 37 degrees aggregate by trajectory
chain_B_sub_37_deg = pd.concat([traj_1_chain_B_sub_37_deg,
                               traj_2_chain_B_sub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))
chain_B_nosub_37_deg = pd.concat([traj_1_chain_B_nosub_37_deg,
                               traj_2_chain_B_nosub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))

In [ ]:
# Aggregate by temp - chain A
chain_A_sub = pd.concat([traj_1_chain_A_sub_30_deg,traj_2_chain_A_sub_30_deg,
                         traj_1_chain_A_sub_37_deg,traj_2_chain_A_sub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'),std=('Area','std'))
chain_A_nosub = pd.concat([traj_1_chain_A_nosub_30_deg,traj_2_chain_A_nosub_30_deg,
                         traj_1_chain_A_nosub_37_deg,traj_2_chain_A_nosub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'),std=('Area','std'))

In [ ]:
# Aggregate by temp - chain B
chain_B_sub = pd.concat([traj_1_chain_B_sub_30_deg,traj_2_chain_B_sub_30_deg,
                         traj_1_chain_B_sub_37_deg,traj_2_chain_B_sub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'),std=('Area','std'))
chain_B_nosub = pd.concat([traj_1_chain_B_nosub_30_deg,traj_2_chain_B_nosub_30_deg,
                         traj_1_chain_B_nosub_37_deg,traj_2_chain_B_nosub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'),std=('Area','std'))

In [ ]:
#Reset index to remove multi-indexing - Chain A 30 degrees
chain_A_sub_30_deg.reset_index(inplace = True)
chain_A_nosub_30_deg.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain B 30 degrees
chain_B_sub_30_deg.reset_index(inplace = True)
chain_B_nosub_30_deg.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain A 37 degrees
chain_A_sub_37_deg.reset_index(inplace = True)
chain_A_nosub_37_deg.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain B 37 degrees
chain_B_sub_37_deg.reset_index(inplace = True)
chain_B_nosub_37_deg.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain A
chain_A_sub.reset_index(inplace = True)
chain_A_nosub.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain B
chain_B_sub.reset_index(inplace = True)
chain_B_nosub.reset_index(inplace = True)

In [ ]:
# Calulate rolling average - Chain A 30 degrees
chain_A_sub_30_deg_rolling = chain_A_sub_30_deg["avg"].rolling(window=1000,center=True).mean()
chain_A_nosub_30_deg_rolling = chain_A_nosub_30_deg["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain B 30 degrees
chain_B_sub_30_deg_rolling = chain_B_sub_30_deg["avg"].rolling(window=1000,center=True).mean()
chain_B_nosub_30_deg_rolling = chain_B_nosub_30_deg["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain A 37 degrees
chain_A_sub_37_deg_rolling = chain_A_sub_37_deg["avg"].rolling(window=1000,center=True).mean()
chain_A_nosub_37_deg_rolling = chain_A_nosub_37_deg["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain B 37 degrees
chain_B_sub_37_deg_rolling = chain_B_sub_37_deg["avg"].rolling(window=1000,center=True).mean()
chain_B_nosub_37_deg_rolling = chain_B_nosub_37_deg["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain A
chain_A_sub_rolling = chain_A_sub["avg"].rolling(window=1000,center=True).mean()
chain_A_nosub_rolling = chain_A_nosub["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain A
chain_B_sub_rolling = chain_B_sub["avg"].rolling(window=1000,center=True).mean()
chain_B_nosub_rolling = chain_B_nosub["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Plot results - Chain A 30 degrees
plt.errorbar(x=chain_A_sub_30_deg["Time(ns)"],y=chain_A_sub_30_deg["avg"],label="Substrate 303.15 K")
plt.errorbar(x=chain_A_nosub_30_deg["Time(ns)"],y=chain_A_nosub_30_deg["avg"],label="No Substrate 303.15 K")
plt.errorbar(x=chain_A_sub_30_deg["Time(ns)"],y=chain_A_sub_30_deg_rolling,label="Rolling Average Substrate")
plt.errorbar(x=chain_A_nosub_30_deg["Time(ns)"],y=chain_A_nosub_30_deg_rolling,label="Rolling Average No Substrate")
plt.legend(loc='best')
plt.title("Chain A Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain A SASA Sub vs No Sub 303.15 K labels.png", dpi=300)

In [ ]:
# Plot results - Chain B 30 degrees
plt.errorbar(x=chain_B_sub_30_deg["Time(ns)"],y=chain_B_sub_30_deg["avg"],label="Substrate 303.15 K")
plt.errorbar(x=chain_B_nosub_30_deg["Time(ns)"],y=chain_B_nosub_30_deg["avg"],label="No Substrate 303.15 K")
plt.errorbar(x=chain_B_sub_30_deg["Time(ns)"],y=chain_B_sub_30_deg_rolling,label="Rolling Average Substrate")
plt.errorbar(x=chain_B_nosub_30_deg["Time(ns)"],y=chain_B_nosub_30_deg_rolling,label="Rolling Average No Substrate")
plt.legend(loc='best')
plt.title("Chain B Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain B SASA Sub vs No Sub 303.15 K labels.png", dpi=300)

In [ ]:
# Plot results - Chain A 37 degrees
plt.errorbar(x=chain_A_sub_37_deg["Time(ns)"],y=chain_A_sub_37_deg["avg"],label="Substrate 310.15 K")
plt.errorbar(x=chain_A_nosub_37_deg["Time(ns)"],y=chain_A_nosub_37_deg["avg"],label="No Substrate 310.15 K")
plt.errorbar(x=chain_A_sub_37_deg["Time(ns)"],y=chain_A_sub_37_deg_rolling,label="Rolling Average Substrate")
plt.errorbar(x=chain_A_nosub_37_deg["Time(ns)"],y=chain_A_nosub_37_deg_rolling,label="Rolling Average No Substrate")
plt.legend(loc='best')
plt.title("Chain A Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain A SASA Sub vs No Sub 310.15 K labels.png", dpi=300)

In [ ]:
# Plot results - Chain B 37 degrees
plt.errorbar(x=chain_B_sub_37_deg["Time(ns)"],y=chain_B_sub_37_deg["avg"],label="Substrate 310.15 K")
plt.errorbar(x=chain_B_nosub_37_deg["Time(ns)"],y=chain_B_nosub_37_deg["avg"],label="No Substrate 310.15 K")
plt.errorbar(x=chain_B_sub_37_deg["Time(ns)"],y=chain_B_sub_37_deg_rolling,label="Rolling Average Substrate")
plt.errorbar(x=chain_B_nosub_37_deg["Time(ns)"],y=chain_B_nosub_37_deg_rolling,label="Rolling Average No Substrate")
plt.legend(loc='best')
plt.title("Chain B Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain B SASA Sub vs No Sub 310.15 K labels.png", dpi=300)

In [ ]:
# Plot results - Chain A
plt.errorbar(x=chain_A_sub["Time(ns)"],y=chain_A_sub["avg"],yerr=chain_A_sub['std'],label="Substrate")
plt.errorbar(x=chain_A_nosub["Time(ns)"],y=chain_A_nosub["avg"],yerr=chain_A_nosub['std'],label="No Substrate")
plt.errorbar(x=chain_A_sub["Time(ns)"],y=chain_A_sub_rolling)
plt.errorbar(x=chain_A_nosub["Time(ns)"],y=chain_A_nosub_rolling)
plt.legend(loc='upper left')
plt.title("Chain A Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain A SASA Sub vs No Sub.png", dpi=300)

In [ ]:
# Plot results - Chain B
plt.errorbar(x=chain_B_sub["Time(ns)"],y=chain_B_sub["avg"],yerr=chain_B_sub['std'],label="Substrate")
plt.errorbar(x=chain_B_nosub["Time(ns)"],y=chain_B_nosub["avg"],yerr=chain_B_nosub['std'],label="No Substrate")
plt.errorbar(x=chain_B_sub["Time(ns)"],y=chain_B_sub_rolling)
plt.errorbar(x=chain_B_nosub["Time(ns)"],y=chain_B_nosub_rolling)
plt.legend(loc='upper left')
plt.title("Chain B Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain B SASA Sub vs No Sub.png", dpi=300)